# import library

In [1]:
import os
import re
import csv
from tqdm import tqdm

from konlpy.tag import Komoran

import pandas as pd
import ast

# get combined_crawling_data by mysql

In [46]:
def get_combined_crawling_data():
    conn = pymysql.connect(
        host = 'localhost',
        port = 33060,
        user = 'data',
        password = 'data',
        database = 'recommend_system',
        charset='utf8mb4'
    )
    
    ccd_query = "SELECT * FROM combined_crawling_data"
    
    ccd_df = pd.read_sql(ccd_query, conn)
    
    conn.close()

    return ccd_df

# skills 추출 후 중복 제거하고 skills_set list 생성

In [33]:
def get_skills_set():
    skills_list = []
    skills_filter = []

    with open("data/combined_crawling_data.csv", encoding="utf-8") as f:
        reader = csv.reader(f)

        for row in reader:
            # row 리스트의 10번째 값을 가져와 string 을 '' 안의 단어들로 나눠서 리스트로 만들기
            skills = re.findall(r"'(.*?)'", row[10])
            skills_upper = [word.upper() for word in skills]

            # 나누어진 리스트를 추가
            skills_list += skills_upper

        skillset = set(skills_list)

    return skillset

In [35]:
skills_set = get_skills_set()
print(len(skills_set))

659


## db 에서 가져오기

In [47]:
import pymysql
import pandas as pd


def get_skills_set():
    skills_list = []
    skills_filter = []
    
    ccd_df = get_combined_crawling_data()
    
    ccd_skills_list = ccd_df['skills'].tolist()
    
    for row in ccd_skills_list:
        # row 리스트의 10번째 값을 가져와 string 을 '' 안의 단어들로 나눠서 리스트로 만들기
        skills = re.findall(r"'(.*?)'", row)
        skills_upper = [word.upper() for word in skills]

        # 나누어진 리스트를 추가
        skills_list += skills_upper

    skills_set = set(skills_list)

    return skills_set

In [48]:
skills_set = get_skills_set()
print(len(skills_set))

/var/folders/vf/p_2t8d9s3dgc1k68_4bwc6r80000gn/T/ipykernel_17663/2245305523.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ccd_df = pd.read_sql(ccd_query, conn)


659


# 리스트를 딕셔너리로 변환 함수

In [7]:
def list_to_dict(lst):
    toDict = dict()

    for item in lst:
        toDict[item.upper()] = 0

    return toDict

# 딕셔너리를 CSV 파일로 저장하는 함수

In [15]:
def save_csv_file(dict, title):
    with open(title, "w", newline="") as file:
        writer = csv.writer(file)
        
        writer.writerow(["Key", "Kalue"])
        
        for k, v in dict.items():
            writer.writerow([k, v])
            
    print("Dictionary keys and values saved to CSV file successfully.")

# 각 채용 링크의 qualification 속 skills_set 정리하기

In [49]:
def get_qualification_skill_list():
    com_set = get_skills_set()
    com_list = list(com_set)
    com_dict = list_to_dict(com_list)
    
    skill_dict = com_dict.copy()
    freq_dict = dict()
    
    with open("data/combined_crawling_data.csv", "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        data = list(reader)
        count = 1
        
        for row in tqdm(data[1:]):
            id = row[0]
            
            # qualifications = 7, preferred = 8, skills = 10
            quali_data = row[7].upper()
            
            # 확인해야하는 qualification data
            phrases = com_list
            
            # remove symbols
            sentence = re.sub(r'[^\w\s]', '', quali_data)
            
            # re.escape 로 특수문자를 벗어나고, | 로 단어 연결
            pattern = r'\b(' + '|'.join(map(re.escape, phrases)) + r')\b'
            
            # 한글 정렬 후 나누기
            komoran = Komoran()
            sentence = komoran.morphs(sentence)
            
            # 한글 정렬 후 나누고 다시 문장으로 합쳐주기
            sentence = " ".join(sentence)
            
            # phrases 에 있는 단어를 가져와서 sentence 속 빈도 확인하기
            matches = re.findall(pattern, sentence)
            
            if count % 500 == 0:
                print(matches)
            count += 1
            
            freq_dict[id] = list(set(matches))
            
    save_csv_file(freq_dict, "data/freq_qual_data.csv")
    return freq_dict

## db

In [ ]:
def get_qualification_skill_list():
    com_set = get_skills_set()
    com_list = list(com_set)
    com_dict = list_to_dict(com_list)
    
    skill_dict = com_dict.copy()
    freq_dict = dict()
    
    with open("data/combined_crawling_data.csv", "r", encoding="utf-8") as f:
        reader = csv.reader(f)
        data = list(reader)
        count = 1
        
        for row in tqdm(data[1:]):
            id = row[0]
            
            # qualifications = 7, preferred = 8, skills = 10
            quali_data = row[7].upper()
            
            # 확인해야하는 qualification data
            phrases = com_list
            
            # remove symbols
            sentence = re.sub(r'[^\w\s]', '', quali_data)
            
            # re.escape 로 특수문자를 벗어나고, | 로 단어 연결
            pattern = r'\b(' + '|'.join(map(re.escape, phrases)) + r')\b'
            
            # 한글 정렬 후 나누기
            komoran = Komoran()
            sentence = komoran.morphs(sentence)
            
            # 한글 정렬 후 나누고 다시 문장으로 합쳐주기
            sentence = " ".join(sentence)
            
            # phrases 에 있는 단어를 가져와서 sentence 속 빈도 확인하기
            matches = re.findall(pattern, sentence)
            
            if count % 500 == 0:
                print(matches)
            count += 1
            
            freq_dict[id] = list(set(matches))
            
    save_csv_file(freq_dict, "data/freq_qual_data.csv")
    return freq_dict

# 각 채용 링크의 preferred 속 skills_set 정리하기

In [22]:
def get_preferred_skill_list():
    com_set = get_skills_set()
    com_list = list(com_set)
    com_dict = list_to_dict(com_list)
    
    skill_dict = com_dict.copy()
    freq_dict = dict()
    
    with open("data/combined_crawling_data.csv", 'r', encoding="utf-8") as f:
        reader = csv.reader(f)
        data = list(reader)
        count = 1
        
        for row in tqdm(data[1:]):
            id = row[0]
            
            # qualifications = 7, preferred = 8, skills = 10
            data = row[8].upper()

            # 확인해야 하는 데이터 (list)
            phrases = com_list 

            # remove symbols
            sentence = re.sub(r'[^\w\s]', '', data)

            # re.escape으로 특수문자 벗어나고, |로 단어 연결
            pattern = r'\b(' + '|'.join(map(re.escape, phrases)) + r')\b'

            # 한글 정렬후 나누기
            komoran = Komoran()
            sentence = komoran.morphs(sentence)

            # 한글 정렬 후 나누고 다시 문장으로 합쳐주기
            sentence = " ".join(sentence)
            
            # phrases에 있는 단어 sentence속 빈도 확인하기
            matches = re.findall(pattern, sentence)
            

            if count % 500 == 0:
                print(matches)
            count += 1

            # dictionary에 하나씩 넣어주기 - set으로 중복 제거
            freq_dict[id] = list(set(matches))

    save_csv_file(freq_dict, "data/freq_pref_data.csv")
    return freq_dict

In [23]:
get_preferred_skill_list()

  1%|▉                                                                                                        | 51/5451 [01:07<1:58:18,  1.31s/it]


KeyboardInterrupt: 

# 각 채용 링크의 skills 속 skills_set 정리하기

In [16]:
def get_skills_skill_list():
    freq_dict = dict()

    with open("data/combined_crawling_data.csv", 'r', encoding="utf-8") as f:
        skill_reader = csv.reader(f)
        skill = list(skill_reader)

        for row in skill[1:]:
            id = row[0]
            
            data = row[10].upper()
            
            # id = 0,  skills = 10
            freq_dict[id] = data

    print(len(freq_dict))

    save_csv_file(freq_dict, "data/freq_skill_data.csv")

In [17]:
get_skills_skill_list()

5450
Dictionary keys and values saved to CSV file successfully.


In [ ]:
def get_skills_include_skill_list():
    freq_dict = dict()

    with open("data/combined_crawling_data.csv", 'r', encoding="utf-8") as f:
        skill_reader = csv.reader(f)
        skill = list(skill_reader)

        for row in skill[1:]:
            id = row[0]
            
            data = row[10].upper()
            
            # id = 0,  skills = 10
            freq_dict[id] = data

    print(len(freq_dict))

    save_csv_file(freq_dict, "data/freq_skill_data.csv")

# 각 리스트에 데이터를 넣어서 리스트 = 컬럼으로 한 csv 파일 작성

In [25]:
total_dict = dict()

ids = []

with open("data/freq_skill_data.csv","r",encoding="utf-8") as skill:
    skill_reader = csv.reader(skill)
    skill_rows = list(skill_reader)
    skills = []

with open("data/freq_qual_data.csv","r",encoding="cp949") as qual:
    qual_reader = csv.reader(qual)
    qual_rows = list(qual_reader)
    quals = []

with open("data/freq_pref_data.csv","r",encoding="cp949") as pref:
    pref_reader = csv.reader(pref)
    pref_rows = list(pref_reader)
    prefs = []

    print(len(skill_rows), len(qual_rows), len(pref_rows))
    count = 0
    for row in range(1, len(skill_rows)):
        if (skill_rows[row][0] == qual_rows[row][0] == pref_rows[row][0]):
            id = skill_rows[row][0]
            skill = skill_rows[row][1]
            qual = qual_rows[row][1]
            pref = pref_rows[row][1]

            # 각 리스트에도 넣어준다.
            ids.append(id)
            skills.append(skill)
            quals.append(qual)
            prefs.append(pref)

        if row % 500 == 0:
            print(skill_rows[row][0], qual_rows[row][0], pref_rows[row][0])
            print(skill_rows[row][1], qual_rows[row][1], pref_rows[row][1])

5451 5451 5451
135926 135926 135926
 ['SQL', '관리'] ['개발', 'ORACLE', 'JAVA', 'MSSQL', 'MYSQL', 'TOMCAT']
149822 149822 149822
 ['개발', 'API', '서버', '설계', 'ELASTICSEARCH', 'SOLR', '검색 엔진'] ['개발', 'KUBERNETES']
80377 80377 80377
 ['QT', 'C'] ['개발', 'VISUAL STUDIO', 'C']
152260 152260 152260
['GIT', 'GITLAB', 'JAVASCRIPT', 'TYPESCRIPT', 'REACT NATIVE', 'REDUX', 'NEXT.JS'] ['개발', 'API', 'TYPESCRIPT', 'REACT NATIVE', 'REST', 'CSS'] ['개발', 'IOS', 'FLOW', 'RESTFUL API', 'GIT', '리팩토링', 'ANDROID', 'REACT']
145706 145706 145706
['MYSQL', 'JAVA', 'KOTLIN', 'JUNIT', 'JPA', 'SPRING BOOT'] ['개발', 'SPRING BOOT', 'KOTLIN', 'ORM', 'GIT', 'JAVA', 'JPA', 'API 개발', 'MSSQL', 'RDBMS', 'MVC', '모델링', '관리'] ['개발', 'DOCKER', '설계', '검증', '최적화', 'NOSQL', 'MYSQL', 'JIRA', 'JUNIT', 'API', '서버', 'MVC', 'SPRING FRAMEWORK', 'SLACK', 'AWS', 'GRADLE', 'CONFLUENCE', 'GIT', 'JAVA', 'JPA', 'GITLAB', 'REDIS', 'JENKINS']
163231 163231 163231
['CISSP', 'CISA'] ['개발', '정보 보안', '보안', '네트워크 보안', '인프라', '보안 운영', '관리'] ['개발', '정보 보안

# quali, pref, skills 데이터 하나로 합치기

In [26]:
# Create the DataFrame
df = pd.DataFrame({'id': ids, 'qualification': quals, 'preferred': prefs, 'skills': skills})

# Save the DataFrame as a CSV file with column names
df.to_csv('data/qual_pref_skill_combined.csv', index=False, encoding="utf-8")